# LOCI Demo Application

In [ ]:
from __future__ import print_function
import ipywidgets as widgets
from IPython.display import display

import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.append('..')

%load_ext autoreload
%autoreload 2

import loci as lc
from loci import io
from loci import analytics
from loci import clustering
from loci import brs
from loci import plots

import geopandas as gpd
import folium
from folium import plugins
from shapely.geometry import Polygon
from statistics import mean, median
import networkx as nx
from random import sample
import time
import math

In [ ]:
## Load data

In [ ]:
gdf = None
kwds_freq = None
description_style = {'description_width': 'initial'}

mbrs_G = None
mbrs_rtree = None

In [ ]:
### WIDGETS FOR 'LOAD DATA' ###

w_input_file = widgets.Text(value='', description='Input file:', style=description_style, placeholder='osmpois-greece.csv')
w_col_sep = widgets.Text(value='', description='Column separator:', style=description_style, placeholder=';')
w_col_id = widgets.Text(value='', description='ID column:', style=description_style, placeholder='id')
w_col_name = widgets.Text(value='', description='Name column:', style=description_style, placeholder='name')
w_col_lon = widgets.Text(value='', description='Longitude column:', style=description_style, placeholder='lon')
w_col_lat = widgets.Text(value='', description='Latitude column:', style=description_style, placeholder='lat')
w_col_kwds = widgets.Text(value='', description='Keywords column:', style=description_style, placeholder='kwds')
w_kwds_sep = widgets.Text(value='', description='Keywords separator:', style=description_style, placeholder=',')
w_source_crs = widgets.Text(value='', description='Source CRS:', style=description_style, placeholder='EPSG:4326')
w_target_crs = widgets.Text(value='', description='Target CRS:', style=description_style, placeholder='EPSG:4326')
w_min_lon = widgets.Text(value='', description='Min lon:', style=description_style, placeholder='23.48')
w_max_lon = widgets.Text(value='', description='Max lon:', style=description_style, placeholder='23.98')
w_min_lat = widgets.Text(value='', description='Min lat:', style=description_style, placeholder='37.83')
w_max_lat = widgets.Text(value='', description='Max lat:', style=description_style, placeholder='38.08')
w_button_load_data = widgets.Button(description='Load', style=description_style)
w_out_load_data = widgets.Output()
ui_load_data = widgets.VBox([w_input_file, w_col_sep, w_col_id, w_col_name, w_col_lon, w_col_lat, w_col_kwds, w_kwds_sep, w_source_crs, w_target_crs, w_min_lon, w_max_lon, w_min_lat, w_max_lat, w_button_load_data, w_out_load_data])

In [ ]:
### FUNCTIONS FOR 'LOAD DATA' ###

def load_data(btn):
    
    global gdf
    global kwds_freq
    
    gdf = lc.io.read_csv(input_file=w_input_file.value,
                         sep=w_col_sep.value,
                         col_id=w_col_id.value,
                         col_name=w_col_name.value,
                         col_lon=w_col_lon.value,
                         col_lat=w_col_lat.value,
                         col_kwds=w_col_kwds.value,
                         kwds_sep=w_kwds_sep.value,
                         source_crs=w_source_crs.value,
                         target_crs=w_target_crs.value)
    
    if w_min_lon.value != '' and w_max_lon.value != '' and w_min_lat.value != '' and w_max_lat.value != '':
        gdf = lc.io.crop(gdf, float(w_min_lon.value), float(w_min_lat.value), float(w_max_lon.value), float(w_max_lat.value))
        gdf.reset_index(drop=True, inplace=True)
    
    with w_out_load_data:
        w_out_load_data.clear_output()
        display(gdf)
    
    kwds_freq = lc.analytics.kwds_freq(gdf)
    kwds_list = [('ALL (' + str(len(gdf.index)) + ')', '---')]
    kwds_list = kwds_list + [((k + ' (' + str(v) + ')'), k) for k, v in kwds_freq.items()]
    w_points_kwd_selection.options = kwds_list
    w_heatmap_kwd_selection.options = kwds_list
    w_clusters_kwd_selection.options = kwds_list
    
    w_points_sample_size.max = len(gdf.index)
    w_points_sample_size.value = round(0.1 * len(gdf.index))
    w_points_sample_size.min = 1
    w_points_sample_size.step = 1
    
    w_heatmap_sample_size.max = len(gdf.index)
    w_heatmap_sample_size.value = round(0.1 * len(gdf.index))
    w_heatmap_sample_size.min = 1
    w_heatmap_sample_size.step = 1
    
    w_clusters_sample_size.max = len(gdf.index)
    w_clusters_sample_size.value = round(0.1 * len(gdf.index))
    w_clusters_sample_size.min = 1
    w_clusters_sample_size.step = 1
        

w_button_load_data.on_click(load_data)

In [ ]:
### WIDGETS FOR 'MAP: POINTS' ###

w_points_kwd_selection = widgets.Dropdown(options=[], description='Select keyword:', style=description_style)
w_points_sample_size = widgets.IntSlider(value=0, min=0, max=0, step=0, description='Sample size:', style=description_style)
w_show_bbox = widgets.Checkbox(value=False, description='Show bbox', style=description_style)
w_button_show_points = widgets.Button(description='Show Points')
w_map_points = widgets.Output(layout={'width': '50%'})
ui_map_points = widgets.VBox([w_points_kwd_selection, w_points_sample_size, w_show_bbox, w_button_show_points, w_map_points])

In [ ]:
### FUNCTIONS FOR 'MAP: POINTS' ###

def show_map_points(btn):
    
    global gdf
    
    if gdf is not None:
        sample_size = w_points_sample_size.value
        kwd = w_points_kwd_selection.value
        
        if kwd == '---':
            kwd = None
        
        show_bbox = w_show_bbox.value
        m = lc.plots.map_points(gdf, sample_size=sample_size, kwd=kwd, show_bbox=show_bbox)

        with w_map_points:
            w_map_points.clear_output()
            display(m)
    else:
        with w_map_points:
            print('No dataset is loaded.')

            
w_button_show_points.on_click(show_map_points)


def update_w_points_sample_size(*args):
    kwd = w_points_kwd_selection.value
    if kwd == '---':
        new_size = len(gdf.index)
    else:
        new_size = kwds_freq[kwd]
    w_points_sample_size.max = new_size
    w_points_sample_size.value = round(0.1 * new_size)
    w_points_sample_size.min = 1
    w_points_sample_size.step = 1


w_points_kwd_selection.observe(update_w_points_sample_size, 'value')

In [ ]:
### WIDGETS FOR 'MAP: HEATMAP' ###

w_heatmap_kwd_selection = widgets.Dropdown(options=[], description='Select keyword:', style=description_style)
w_heatmap_sample_size = widgets.IntSlider(value=0, min=0, max=0, step=0, description='Sample size:', style=description_style)
w_heatmap_radius = widgets.IntSlider(value=10, min=1, max=50, step=1, description='Radius:', style=description_style)
w_button_show_heatmap = widgets.Button(description='Show Heatmap')
w_map_heatmap = widgets.Output(layout={'width': '50%'})
ui_map_heatmap = widgets.VBox([w_heatmap_kwd_selection, w_heatmap_sample_size, w_heatmap_radius, w_button_show_heatmap, w_map_heatmap])

In [ ]:
### FUNCTIONS FOR 'MAP: HEATMAP' ###

def show_map_heatmap(btn):
    
    global gdf
    
    if gdf is not None:
        sample_size = w_heatmap_sample_size.value
        kwd = w_heatmap_kwd_selection.value
        
        if kwd == '---':
            kwd = None
        
        radius = w_heatmap_radius.value
        
        m = lc.plots.heatmap(gdf, sample_size=sample_size, kwd=kwd, radius=radius)

        with w_map_heatmap:
            w_map_heatmap.clear_output()
            display(m)
    else:
        with w_map_heatmap:
            print('No dataset is loaded.')

            
w_button_show_heatmap.on_click(show_map_heatmap)


def update_w_heatmap_sample_size(*args):
    kwd = w_heatmap_kwd_selection.value
    if kwd == '---':
        new_size = len(gdf.index)
    else:
        new_size = kwds_freq[kwd]
    w_heatmap_sample_size.max = new_size
    w_heatmap_sample_size.value = round(0.1 * new_size)
    w_heatmap_sample_size.min = 1
    w_heatmap_sample_size.step = 1


w_heatmap_kwd_selection.observe(update_w_heatmap_sample_size, 'value')

In [ ]:
### WIDGETS FOR 'MAP: DENSITY CLUSTERS' ###

w_clusters_kwd_selection = widgets.Dropdown(options=[], description='Select keyword:', style=description_style)
w_clusters_sample_size = widgets.IntSlider(value=0, min=0, max=0, step=0, description='Sample size:', style=description_style)
w_clusters_alg = widgets.Dropdown(options=['DBSCAN', 'HDBSCAN'], description='Select algorithm:', style=description_style)
w_clusters_min_pts = widgets.IntText(value=100, min=1, step=1, description='Min Size: ', style=description_style)
w_clusters_eps = widgets.BoundedFloatText(value=0.001, min=0.001, step=0.001, description='Radius: ', style=description_style)
w_button_show_clusters = widgets.Button(description='Compute Clusters')
w_map_clusters = widgets.Output(layout={'width': '50%'})
ui_map_clusters = widgets.VBox([w_clusters_kwd_selection, w_clusters_sample_size, w_clusters_alg, w_clusters_min_pts, w_clusters_eps, w_button_show_clusters, w_map_clusters])

In [ ]:
### FUNCTIONS FOR 'MAP: CLUSTERS' ###

def show_map_clusters(btn):
    
    global gdf
    
    if gdf is not None:
        sample_size = w_clusters_sample_size.value
        kwd = w_clusters_kwd_selection.value
        
        if kwd == '---':
            kwd = None
        
        min_pts = w_clusters_min_pts.value
        eps = w_clusters_eps.value
        alg = w_clusters_alg.value
        pois_in_clusters, eps_per_cluster = lc.clustering.compute_clusters(gdf,
                                                                   alg=alg,
                                                                   min_pts=min_pts,
                                                                   eps=eps,
                                                                   sample_size=sample_size,
                                                                   kwd=kwd,
                                                                   n_jobs=-1)
        cluster_borders = lc.clustering.cluster_shapes(pois_in_clusters, 3, eps_per_cluster)
        if len(cluster_borders.index) < 2:
            return
        m = lc.plots.map_choropleth(cluster_borders, id_field='cluster_id', value_field='size')

        with w_map_clusters:
            w_map_clusters.clear_output()
            display(m)
    else:
        with w_map_clusters:
            print('No dataset is loaded.')

            
w_button_show_clusters.on_click(show_map_clusters)


def update_w_clusters_sample_size(*args):
    kwd = w_clusters_kwd_selection.value
    if kwd == '---':
        new_size = len(gdf.index)
    else:
        new_size = kwds_freq[kwd]
    w_clusters_sample_size.max = new_size
    w_clusters_sample_size.value = round(0.1 * new_size)
    w_clusters_sample_size.min = 1
    w_clusters_sample_size.step = 1


w_clusters_kwd_selection.observe(update_w_clusters_sample_size, 'value')

In [ ]:
### WIDGETS FOR 'MAP: MIXTURE CLUSTERS' ###

# create graph
w_mbrs_eps = widgets.BoundedFloatText(value=0.001, min=0.001, step=0.001, description='Radius: ', style=description_style)
w_button_mbrs_graph = widgets.Button(description='Create Graph')

# compute regions
w_mbrs_max_size = widgets.BoundedIntText(value=100, min=2, max=500, step=1, description='Max size: ', style=description_style)
w_mbrs_size_weight = widgets.BoundedFloatText(value=0.1, min=0, step=0.01, description='Size weight: ', style=description_style)
w_mbrs_time_budget = widgets.BoundedIntText(value=30, min=1, step=1, description='Time budget (sec): ', style=description_style)
w_mbrs_entropy_mode = widgets.Dropdown(options=['high', 'low'], description='Entropy mode: ', style=description_style)
w_mbrs_method = widgets.Dropdown(options=['ExpAll', 'ExpSingle', 'ExpHybrid'], description='Method:', style=description_style)
w_mbrs_seeds_ratio = widgets.BoundedIntText(value=1, min=1, max=100, step=1, description='Initial seeds (%): ', style=description_style)

w_button_show_mbrs = widgets.Button(description='Compute Regions')
w_map_mbrs = widgets.Output(layout={'width': '50%'})
ui_map_mbrs = widgets.VBox([w_mbrs_eps, w_button_mbrs_graph, w_mbrs_max_size, w_mbrs_size_weight, w_mbrs_time_budget, w_mbrs_entropy_mode, w_mbrs_method, w_mbrs_seeds_ratio, w_button_show_mbrs, w_map_mbrs])

In [ ]:
### FUNCTIONS FOR 'MAP: MIXTURE CLUSTERS' ###

def mbrs_graph(btn):
    
    global gdf
    global mbrs_G
    global mbrs_rtree
    
    if gdf is not None:
        
        with w_map_mbrs:
            w_map_mbrs.clear_output()
            print('Creating spatial connectivity graph...')
        
        eps = w_mbrs_eps.value
        mbrs_G, mbrs_rtree = lc.brs.create_graph(gdf, eps)
        
        cc = [d for n, d in mbrs_G.degree()]

        with w_map_mbrs:
            print('Graph created successfully.')
            # check max node degree
            max_degree = sorted(cc)[-1] + 1
            mean_degree = mean(cc)
            median_degree = median(cc)
            print('Max degree: ' + str(max_degree) + ' Mean degree: ' + str(mean_degree) + ' Median degree: ' + str(median_degree))
    
            # check connected components
            print('Max connected component: ' + str([len(c) for c in sorted(nx.connected_components(mbrs_G), key=len, reverse=True)][0]))
    else:
        with w_map_mbrs:
            print('No dataset is loaded.')

            
def mbrs_regions(btn):
    
    global gdf
    global mbrs_G
    global mbrs_rtree
    
    if gdf is not None and mbrs_G is not None:
        seeds_ratio = int(w_mbrs_seeds_ratio.value) / 100
        types, colors = lc.brs.get_types(gdf)
        
        params = {
            'variables': {
                'max_size': {'current': w_mbrs_max_size.value},
                'size_weight': {'current': w_mbrs_size_weight.value},
                'time_budget': {'current': w_mbrs_time_budget.value}
            },
            'methods': {'current': w_mbrs_method.value},
            'entropy_mode': {'current': w_mbrs_entropy_mode.value},
            'settings': {'top_k': 1, 'max_se': math.log(len(types))}
        }
        
        seeds = sample(list(mbrs_G.nodes), int(seeds_ratio * len(list(mbrs_G.nodes))))
        start_time = time.time()
        score, region, updates = lc.brs.run_mbrs(mbrs_G, mbrs_rtree, types, params, start_time, seeds)
        elapsed = time.time() - start_time
        
        m = lc.brs.show_map(gdf, region, colors)
        with w_map_mbrs:
            w_map_mbrs.clear_output()
            display(m)
    
    else:
        with w_map_mbrs:
            print('No dataset or graph is loaded.')

            
w_button_mbrs_graph.on_click(mbrs_graph)

w_button_show_mbrs.on_click(mbrs_regions)

In [ ]:
### TABS ###

tab_explore = widgets.Tab()
tab_explore.children = [ui_map_points, ui_map_heatmap, ui_map_clusters, ui_map_mbrs]
tab_explore.set_title(0, 'Points')
tab_explore.set_title(1, 'Heatmap')
tab_explore.set_title(2, 'Density Clusters')
tab_explore.set_title(3, 'Mixture Clusters')

tab_main = widgets.Tab()
tab_main.children = [ui_load_data, tab_explore]
tab_main.set_title(0, 'Load')
tab_main.set_title(1, 'Explore')
display(tab_main)